<a href="https://colab.research.google.com/github/pravsels/DistilLM/blob/main/HF_Training_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers accelerate peft bitsandbytes

In [2]:
! apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
import transformers
from datasets import load_dataset
import math
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

In [5]:
wikitext_dataset = load_dataset('mediciresearch/manimation')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
wikitext_dataset['train'][11]

{'text': '<s>[INST] Generate accurate and correct ManimCE Python code for the animation requested by the user. Here is the user\'s request: I would like to create an educational math animation that explains the concept of Fourier Series with voiceover using a simple and elegantly animated whiteboard style presentation. The animation should start by stating "We know that the basic trigonometric series is...", followed by the appearance of the corresponding mathematical equation. Then, it should explain that this becomes a Fourier Series when specific terms are substituted with particular integral expressions, and it should display these integrals clearly.\n\nThe important elements of the animation include:\n- High-quality text and equations that are clear and easy to read.\n- The use of engaging voiceover with a natural English accent, preferable male, that fluidly narrates the information as it appears on screen.\n- The voiceover should have a style that resembles a casual but authorit

In [7]:
model_name = 'deepseek-ai/deepseek-coder-6.7b-instruct'
# model_name = 'smallcloudai/Refact-1_6B-fim'
# using the same tokenizer that was used for training the model
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [8]:
def tokenize_func(examples):
  return tokenizer(examples['text'])

In [9]:
tokenized_dataset = wikitext_dataset.map(tokenize_func,
                                         batched=True,
                                         num_proc=8,
                                         remove_columns=['text'])

In [10]:
print(tokenized_dataset)
print(tokenized_dataset['train'][5])

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1023
    })
})
{'input_ids': [32013, 27, 82, 29, 58, 24607, 60, 9338, 387, 10370, 285, 2907, 2797, 308, 4402, 13003, 2974, 327, 254, 17181, 13159, 457, 254, 2664, 13, 4451, 317, 254, 2664, 6, 82, 3092, 25, 303, 740, 833, 276, 2594, 274, 14711, 17181, 1064, 245, 24222, 3857, 6267, 331, 254, 4141, 365, 4761, 8719, 13, 7925, 1208, 1020, 330, 4640, 891, 1372, 4075, 11, 285, 1317, 4075, 1020, 463, 274, 3750, 1594, 5869, 473, 207, 17, 285, 6733, 457, 207, 16, 327, 1317, 15547, 4075, 13, 428, 19636, 1020, 330, 3286, 271, 276, 254, 1327, 280, 1317, 4075, 13, 303, 927, 254, 17181, 276, 9411, 2288, 578, 437, 3857, 331, 12, 11254, 11, 3939, 782, 207, 23, 7665, 276, 5582, 3786, 13, 2744, 340, 2594, 1109, 274, 17181, 327, 523, 11, 2837, 254, 4761, 440, 42, 2558, 950, 440, 45, 14289, 65, 950, 285, 440, 3098, 6450, 10823, 428, 14293, 1020, 330, 8493, 326, 24222, 473, 254, 2422, 285, 254, 19636, 13,

In [11]:
# setting context length of out inputs
# block_size = tokenizer.model_max_length
block_size = 1024     # context len of input

In [12]:
def group_text_func(examples):
  # for every key ('train', 'test' etc.), concatenate the texts
  concat_examples = {key: sum(examples[key], []) for key in examples.keys()}
  total_len = len(concat_examples[list(examples.keys())[0]])
  total_len = (total_len // block_size) * block_size

  # split by chunks of block_size
  result = {
      key: [text[i : i + block_size] for i in range(0, total_len, block_size)] for key, text in concat_examples.items()
  }
  result['labels'] = result['input_ids'].copy()

  return result

In [13]:
lm_dataset = tokenized_dataset.map(group_text_func,
                                   batched=True,
                                   batch_size=512,
                                   num_proc=8)
lm_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 635
    })
})

In [14]:
print('input ids : ', lm_dataset['train'][0]['input_ids'][:5])
print('attention mask : ', lm_dataset['train'][0]['attention_mask'][:5])
print('label ids : ', lm_dataset['train'][0]['labels'][:5])

input ids :  [32013, 27, 82, 29, 58]
attention mask :  [1, 1, 1, 1, 1]
label ids :  [32013, 27, 82, 29, 58]


In [15]:
from peft import LoraConfig

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj"],
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [16]:
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             load_in_8bit=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
model.add_adapter(peft_config,
                  adapter_name="manim_adapter")

In [18]:
import numpy as np
def count_trainable_parameters(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    total_params = sum([np.prod(p.size()) for p in model.parameters()])
    return params/total_params

count_trainable_parameters(model) * 100

0.49533656825843125

In [19]:
training_args = TrainingArguments(
  f"{model_name}-finetuned-manimation",
  evaluation_strategy = "epoch",
  learning_rate=2e-5,
  per_device_train_batch_size=1,
  per_device_eval_batch_size=1,
  gradient_accumulation_steps=16,
  weight_decay=0.01,
  push_to_hub=True
)

In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset['train'],
    # eval_dataset=lm_dataset['validation']
)

In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss


ValueError: Trainer: evaluation requires an eval_dataset.

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity score : {math.exp(eval_results['eval_loss']):.2f}")

In [26]:
gen_context_len = 4096

input_text = "write manim code to transition a blue circle to a red square."
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_length=gen_context_len)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


KeyboardInterrupt: 

In [ ]:
trainer.push_to_hub()

In [ ]:
# load_model_hub = AutoModelForCausalLM.from_pretrained("pravsels/distilgpt2-finetuned-wikitext2")

In [ ]:
# from transformers import DataCollatorForLanguageModeling, AutoModelForMaskedLM

# # masked language modelling
# model_name = "distilroberta-base"

# tokenizer = AutoTokenizer.from_pretrained(model_name,
#                                           use_fast=True)

# tokenized_dataset = wikitext_dataset.map(tokenize_func,
#                                          batched=True,
#                                          num_proc=8,
#                                          remove_columns=["text"])



# lm_dataset = tokenized_dataset.map(
#     group_text_func,
#     batched=True,
#     batch_size=512,
#     num_proc=8
# )

# model = AutoModelForMaskedLM.from_pretrained(model_name)

# training_args = TrainingArguments(
#     f"{model_name}-finetuned-wikitext2",
#     evaluation_strategy = "epoch",
#     learning_rate=2e-5,
#     weight_decay=0.01,
#     push_to_hub=True,
# )

# # we do this for random masking of tokens
# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,
#                                                 mlm_probability=0.15)


# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=lm_dataset["train"],
#     eval_dataset=lm_dataset["validation"],
#     data_collator=data_collator,
# )

# trainer.train()

In [ ]:
# eval_results = trainer.evaluate()
# print(f"Perplexity score : {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
# trainer.push_to_hub()